In [148]:
import pandas as pd
import numpy as np
import zipfile

!pip install nltk
import re
import nltk 
import string
from markdown import Markdown
from io import StringIO

!pip install vaderSentiment

from openai import OpenAI

In [32]:
client = OpenAI(api_key="...")

# text = "I am bullish on $AAPL"
# response = client.completions.create(model="gpt-3.5-turbo",
# prompt=(f"Sentiment analysis of the following text: '{text}'\n\nSentiment score: "),
# temperature=0,
# max_tokens=1)

# sentiment = response.choices[0].text.strip()
# print(sentiment)

In [134]:
response = client.chat.completions.create(
        model="gpt-4",
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": "As an AI with expertise in language and emotion analysis, your task is to analyze the sentiment of the following comment. This comment was posted on the r/nfl subreddit after a regular season NFL match. Please consider the overall tone of the discussion, the emotion conveyed by the language used, and the context in which words and phrases are used. Use one word to indicate whether the sentiment is generally positive, negative, or neutral."
            },
            {
                "role": "user",
                "content": clean_text(texts[0])
            }
        ],
        max_tokens=1
    )

In [135]:
response

ChatCompletion(id='chatcmpl-979t4oSZJzFu7WiL0bripWaUeX02J', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='Positive', role='assistant', function_call=None, tool_calls=None))], created=1711493330, model='gpt-4-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=1, prompt_tokens=105, total_tokens=106))

In [114]:
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
) 

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def sentiment_analysis(transcription):
    response = client.chat.completions.create(
        model="gpt-4",
        temperature=0,
        messages=[
            {
                "role": "system",
                "content": "As an AI with expertise in language and emotion analysis, your task is to analyze the sentiment of the following comment. This comment was posted on the r/nfl subreddit after a regular season NFL match. Please consider the overall tone of the discussion, the emotion conveyed by the language used, and the context in which words and phrases are used. Use one word to indicate whether the sentiment is generally positive, negative, or neutral."
            },
            {
                "role": "user",
                "content": transcription
            }
        ],
        max_tokens=1
    )
    return response.choices[0].message.content

sentiment_analysis("I don't want to hear another pathetic word about how much the Packers are getting \"helped,\" get over yourselves")


'Negative'

In [42]:
# transcription = "I don't want to hear another pathetic word about how much the Packers are getting \"helped,\" get over yourselves"
# response = client.chat.completions.create(
#         model="gpt-3.5-turbo",
#         temperature=0,
#         messages=[
#             {
#                 "role": "system",
#                 "content": "As an AI with expertise in language and emotion analysis, your task is to analyze the sentiment of the following comment. This comment was posted on Reddit after a regular season NFL match. Please consider the overall tone of the discussion, the emotion conveyed by the language used, and the context in which words and phrases are used. Use one word to indicate whether the sentiment is generally positive, negative, or neutral."
#             },
#             {
#                 "role": "user",
#                 "content": transcription
#             }
#         ],
#         max_tokens=1
#     )

In [45]:
response.choices[0].message.content

'Negative'

# Loading in data

In [51]:
with zipfile.ZipFile("../data/reddit_scrape.csv.zip") as z:
    with z.open("reddit_scrape.csv") as f:
        reddit_df = pd.read_csv(f)

reddit_df['Date'] = pd.to_datetime(reddit_df['Date'])    
reddit_df.head()

scrape_cols = [x for x in reddit_df.columns if 'craped' in x]
for col in scrape_cols: # converts strings to lists
    reddit_df[col] = reddit_df[col].apply(lambda x: eval(x) if type(x)==str else x)

    
reddit_df['game_thread_comments'] = [lst[1:] for lst in reddit_df.loc[:,'Game thread scraped']]
reddit_df['post_game_thread_comments'] = [lst[1:] if type(lst)==list else lst for lst in reddit_df.loc[:,'Post game thread Scraped']]
reddit_df['all comments'] =  reddit_df['game_thread_comments'] + reddit_df['post_game_thread_comments']
reddit_df.loc[reddit_df['all comments'].isna(),'all comments'] = \
        [lst[1:] for lst in reddit_df.loc[reddit_df['all comments'].isna(),'Game thread scraped']]
reddit_df.drop(columns=['game_thread_comments','post_game_thread_comments'],inplace=True)


/opt/homebrew/Caskroom/miniforge/base/envs/sklearn-env/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


In [52]:
def unmark_element(element, stream=None):
    if stream is None:
        stream = StringIO()
    if element.text:
        stream.write(element.text)
    for sub in element:
        unmark_element(sub, stream)
    if element.tail:
        stream.write(element.tail)
    return stream.getvalue()


# patching Markdown
Markdown.output_formats["plain"] = unmark_element
__md = Markdown(output_format="plain")
__md.stripTopLevelTags = False


def unmark(text):
    return __md.convert(text).replace(" \n"," ")

text = reddit_df['Game thread scraped'][0][1][0]
text = unmark(text).replace(" \n"," ")
text

'Dear Falcons fans: are you guys ready for 17 straight weeks of hearing how you fucked up the Super Bowl? I feel your pain. Sincerely, Seahawks fans everywhere.'

In [53]:
text = text.translate(str.maketrans('', '', string.punctuation))
text

'Dear Falcons fans are you guys ready for 17 straight weeks of hearing how you fucked up the Super Bowl I feel your pain Sincerely Seahawks fans everywhere'

In [54]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/rohanjha/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [55]:
def convert_super_bowl(text):
    # Use regular expression to find and replace "super bowl" with "superbowl" (case-insensitive)
    modified_text = re.sub(r'\b(super)\s+(bowl)\b', r'\1\2', text, flags=re.IGNORECASE)
    return modified_text

# Example usage:
original_text = "The Super Bowl is a great event."
converted_text = convert_super_bowl(original_text)
print(converted_text)


The SuperBowl is a great event.


In [107]:
def clean_text(sentence):
    text = unmark(sentence).replace(" \n"," ")
    text = convert_super_bowl(text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text

In [105]:
def sentiment_scores(sentence):
    print(sentence)
    # Create a SentimentIntensityAnalyzer object.
    print("________________________")
    text = unmark(sentence).replace(" \n"," ")
    text = convert_super_bowl(text)
    text = text.translate(str.maketrans('', '', string.punctuation))
    from nltk.tokenize import word_tokenize, RegexpTokenizer
    from nltk.sentiment.vader import SentimentIntensityAnalyzer
    tokenized_sentence = nltk.word_tokenize(text)
    sid_obj = SentimentIntensityAnalyzer()
    pos_word_list=[]
    neu_word_list=[]
    neg_word_list=[]

    for word in tokenized_sentence:
        if (sid_obj.polarity_scores(word)['compound']) >= 0.05:
            pos_word_list.append(word)
        elif (sid_obj.polarity_scores(word)['compound']) <= -0.05:
            neg_word_list.append(word)
        else:
            neu_word_list.append(word)                

    print('Positive:', pos_word_list) 
    # pos_words.append(pos_word_list)
    print('Neutral:', neu_word_list) 
    # neu_words.append(neu_word_list)
    print('Negative:', neg_word_list) 
    # neg_words.append(neg_word_list)
    # polarity_scores method of SentimentIntensityAnalyzer
    # oject gives a sentiment dictionary.
    # which contains pos, neg, neu, and compound scores.
    print("________________________")
    sentiment_dict = sid_obj.polarity_scores(sentence)
    print("Overall sentiment dictionary is : ", sentiment_dict)
    print("sentence was rated as ", sentiment_dict['neg']*100, "% Negative")
    print("sentence was rated as ", sentiment_dict['neu']*100, "% Neutral")
    print("sentence was rated as ", sentiment_dict['pos']*100, "% Positive")
    print("Sentence Overall Rated As", end = " ")
    # decide sentiment as positive, negative and neutral
    if sentiment_dict['compound'] >= 0.05 :
        print("Positive")
        return "Positive"
    elif sentiment_dict['compound'] <= - 0.05 :
        print("Negative")
        return "Negative"
    else :
        print("Neutral")
        return "Neutral"
    
    
    
    

In [73]:
reddit_df['Game thread scraped']

0       [3193, (Dear Falcons fans: are you guys ready ...
1       [109, (I'm a Jets fan. I'm a Texas A&M fan. Ye...
2       [321, (Wow this national anthem singer is a fu...
3       [192, (Man I hope Woodhead stays healthy for t...
4       [367, (1 drive no turnovers. WE'VE TURNED THE ...
                              ...                        
1562    [58, (As long as the Cowboys lose everything i...
1563    [46, (Amazon next gen stats showing that AJ re...
1564    [115, (There's more Lions fans in here than Ra...
1565    [49, (The "bad throw PI" is such fucking bulls...
1566    [1158, (I feel like the NBC pregame show is ba...
Name: Game thread scraped, Length: 1567, dtype: object

In [102]:
import random
num_1 = random.randrange(len(reddit_df))
num_2 = random.randrange(1,20)
text = reddit_df['Game thread scraped'][num_1][1][0]

In [106]:
sentiment_scores(text)

HOLY FUCK, THIS GUY COULDN'T HIT ANYTHING IN A TAMPA JERSEY WHEN HE PLAYED FOR THEM NOW HE CAN'T FUCKING MISS.
________________________
Positive: ['PLAYED']
Neutral: ['HOLY', 'THIS', 'GUY', 'COULDNT', 'HIT', 'ANYTHING', 'IN', 'A', 'TAMPA', 'JERSEY', 'WHEN', 'HE', 'FOR', 'THEM', 'NOW', 'HE', 'CANT', 'FUCKING']
Negative: ['FUCK', 'MISS']
________________________
Overall sentiment dictionary is :  {'neg': 0.143, 'neu': 0.692, 'pos': 0.165, 'compound': -0.1127}
sentence was rated as  14.299999999999999 % Negative
sentence was rated as  69.19999999999999 % Neutral
sentence was rated as  16.5 % Positive
Sentence Overall Rated As Negative


'Negative'

In [104]:
sentiment_analysis(text)

'Negative'

# Testing

In [115]:
import sys, os

def blockPrint():
    sys.stdout = open(os.devnull, 'w')
def enablePrint():
    sys.stdout = sys.__stdout__

texts = []
vader_score = []
gpt_score = []
for i in range(1000):
    num_1 = random.randrange(len(reddit_df))
    num_2 = random.randrange(1,20)
    text = reddit_df['Game thread scraped'][num_1][num_2][0]
    texts.append(text)
    text = clean_text(text)
    blockPrint()
    vader_score.append(sentiment_scores(text))
    enablePrint()
    gpt_score.append(sentiment_analysis(text))

In [122]:
test_df = pd.DataFrame(data=texts,columns=['Texts'])
test_df['vader'] = vader_score
test_df['gpt'] = gpt_score
test_df

,Texts,vader,gpt
0,The Purdy-Kittle connection is something special.,Positive,Positive
1,Super bad play... That could have broken his neck,Negative,Negative
2,SCARY TERRY HALLLOWEEN SPECIAL BITCHES,Negative,Positive
3,This would be a great game in like any other s...,Positive,Negative
4,CRIPPLE FIIIIIIIGHT,Neutral,Negative
...,...,...,...
995,Aight peace y’all ✌️,Positive,Neutral
996,JASON NOOO,Neutral,Negative
997,"Everyone: We are down going into the half, we ...",Positive,Positive
998,Which Seahawks come out today?\nNeck and neck ...,Negative,Negative


In [124]:
test_df.to_csv("gpt_test.csv")

In [140]:
with zipfile.ZipFile("../data/reddit_scrape.csv.zip") as z:
    with z.open("reddit_scrape.csv") as f:
        reddit_df = pd.read_csv(f)

reddit_df['Date'] = pd.to_datetime(reddit_df['Date'])    
reddit_df.head()

scrape_cols = [x for x in reddit_df.columns if 'craped' in x]
for col in scrape_cols: # converts strings to lists
    reddit_df[col] = reddit_df[col].apply(lambda x: eval(x) if type(x)==str else x)

    
reddit_df['game_thread_comments'] = [lst[1:] for lst in reddit_df.loc[:,'Game thread scraped']]
reddit_df['post_game_thread_comments'] = [lst[1:] if type(lst)==list else lst for lst in reddit_df.loc[:,'Post game thread Scraped']]
reddit_df['all comments'] =  reddit_df['game_thread_comments'] + reddit_df['post_game_thread_comments']
reddit_df.loc[reddit_df['all comments'].isna(),'all comments'] = \
        [lst[1:] for lst in reddit_df.loc[reddit_df['all comments'].isna(),'Game thread scraped']]
reddit_df.drop(columns=['game_thread_comments','post_game_thread_comments'],inplace=True)


/opt/homebrew/Caskroom/miniforge/base/envs/sklearn-env/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim
